# BERT

Dans ce notebook on utilise les classes de PyTorch `TransformerEncoderLayer`, `TransformerEncoder` pour implémenter un modèle BERT pour la reconnaissance d'entités nommées.

## Préparer les données

In [1]:
#!pip install torchtext==0.6.0

import time
import torch
from torch import nn
from torch.optim import Adam
from torchtext.data import Field, NestedField, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from torchtext.vocab import Vocab
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np
import pandas as pd
import math
import time
import random
import string
from itertools import chain

import numpy as np
import copy
from copy import deepcopy
import torch.optim as optim
from torch.nn import functional as F

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(lower = True) 
TAG = data.Field(unk_token = None) 
CHAR_NESTING= Field(tokenize=list)
CHAR = NestedField(CHAR_NESTING) 

train_data, valid_data, test_data = data.TabularDataset.splits(
        path="data_ner/",
        train="train.csv",
        validation="valid.csv",
        test="test.csv", format='csv', skip_header=True,
        fields=(
            (("text", "char"), (TEXT, CHAR)), 
            ("tag", TAG)
        )
    )

MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ, # les mots qui apparaissent moins que MIN_FREQ fois seront ignorés du vocabulaire
                 vectors = "glove.6B.300d",
                 unk_init = torch.Tensor.normal_)


TAG.build_vocab(train_data)
CHAR.build_vocab(train_data) 
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort=False)

# padding index
TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
CHAR_PAD_IDX = CHAR.vocab.stoi[CHAR.pad_token]  
TAG_PAD_IDX = TAG.vocab.stoi[TAG.pad_token]

## Construire le modèle

Le modèle BERT est décrit dans cet article et est composé de N = 12 transformers composés de 6 encoders et d'une couche linéaire.

Nous utilisons les paramètres du modèle BERT de l'article $BERT_{BASE}$ avec : 
 
 - N = 12 transformers 
 - Dimension des états cachés : fc_hidden = 768 
 - Têtes d'attention : attn_heads = 12
 
La fonction `clones` suivante permet de produire N couches identiques et sera utile pour assembler N=12 modèles transformers.

In [3]:
def clones(module, N):
    "Produit N couches identiques"
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

On implémente le même modèle transformer que dans un précédent notebook dans une classe à part.

In [36]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=500):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class Transformer(nn.Module):
    def __init__(self, embedding_dim, char_emb_dim, char_cnn_filter_num, attn_heads, dropout, trf_layers,fc_hidden, output_dim):
        super(Transformer, self).__init__()
        
        all_emb_size = embedding_dim + (char_emb_dim * char_cnn_filter_num) 
        self.position_encoder = PositionalEncoding(
            d_model=all_emb_size
        )
        
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=all_emb_size,
            nhead=attn_heads,
            activation="relu",
            dropout=dropout
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layers,
            num_layers=trf_layers
        )
 
                
        self.fc1 = nn.Linear(
        in_features=all_emb_size,
        out_features=fc_hidden
        )
        self.fc1_gelu = nn.GELU()
        self.fc1_norm = nn.LayerNorm(fc_hidden)
        self.fc2_dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(
            in_features=fc_hidden,
            out_features=output_dim
        ) 
    def forward(self, words, word_features, word_pad_idx):
        
        key_padding_mask = torch.as_tensor(words == word_pad_idx).permute(1, 0)
        # pos_out = [sentence length, batch size, embedding dim + char emb dim * num filter]
        pos_out = self.position_encoder(word_features)
        # enc_out = [sentence length, batch size, embedding dim + char emb dim * num filter]
        enc_out = self.encoder(pos_out, src_key_padding_mask=key_padding_mask)
        # Fully-connected
        # fc1_out = [sentence length, batch size, fc hidden]
        fc1_out = self.fc1_norm(self.fc1_gelu(self.fc1(enc_out)))
        # fc2_out = [sentence length, batch size, output dim]
        fc2_out = self.fc2(self.fc2_dropout(fc1_out))
        return fc2_out


  

La classe BERT contient alors l'embedding des mots et des caractères et l'assemblage des N transformers.

In [37]:

class BERT(nn.Module):

    def __init__(self,
                 input_dim,
                 embedding_dim,
                 char_emb_dim,
                 char_input_dim,
                 char_cnn_filter_num,
                 char_cnn_kernel_size,
                 attn_heads,
                 fc_hidden,
                 trf_layers,
                 output_dim,
                 dropout,
                 word_pad_idx,
                 char_pad_idx,
                 tag_pad_idx,
                 N):  
        super().__init__()
        self.N = N
        self.char_pad_idx = char_pad_idx
        self.word_pad_idx = word_pad_idx
        self.tag_pad_idx = tag_pad_idx
        # LAYER 1A: Word Embedding
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(
            num_embeddings=input_dim,
            embedding_dim=embedding_dim,
            padding_idx=word_pad_idx
        )
        self.emb_dropout = nn.Dropout(dropout)
        # LAYER 1B: Char Embedding-CNN
        self.char_emb_dim = char_emb_dim
        self.char_emb = nn.Embedding(
            num_embeddings=char_input_dim,
            embedding_dim=char_emb_dim,
            padding_idx=char_pad_idx
        )
        self.char_cnn = nn.Conv1d(
            in_channels=char_emb_dim,
            out_channels=char_emb_dim * char_cnn_filter_num,
            kernel_size=char_cnn_kernel_size,
            groups=char_emb_dim  # different 1d conv for each embedding dim
        )
        self.cnn_dropout = nn.Dropout(dropout)
        self.layers = clones(Transformer(embedding_dim, char_emb_dim, char_cnn_filter_num, attn_heads, dropout, trf_layers,fc_hidden, output_dim), N)
        #self.trans = Transformer(embedding_dim, char_emb_dim, char_cnn_filter_num, attn_heads, trf_dropout, trf_layers,fc_hidden, output_dim)

        self.softmax = nn.Softmax(dim=1)


        
          
        
        
        # init weights from normal distribution
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1)

    def forward(self, words, chars, tags=None):
        # words = [sentence length, batch size]
        # chars = [batch size, sentence length, word length)
        # tags = [sentence length, batch size]
        # embedding_out = [sentence length, batch size, embedding dim]
        embedding_out = self.emb_dropout(self.embedding(words))
        # character cnn layer forward
        # reference: https://github.com/achernodub/targer/blob/master/src/layers/layer_char_cnn.py
        # char_emb_out = [batch size, sentence length, word length, char emb dim]
        char_emb_out = self.emb_dropout(self.char_emb(chars))
        batch_size, sent_len, word_len, char_emb_dim = char_emb_out.shape
        char_cnn_max_out = torch.zeros(batch_size, sent_len, self.char_cnn.out_channels) 
        for sent_i in range(sent_len):
            # sent_char_emb = [batch size, word length, char emb dim]
            sent_char_emb = char_emb_out[:, sent_i, :, :]
            # sent_char_emb_p = [batch size, char emb dim, word length]
            sent_char_emb_p = sent_char_emb.permute(0, 2, 1)
            # char_cnn_sent_out = [batch size, out channels * char emb dim, word length - kernel size + 1]
            char_cnn_sent_out = self.char_cnn(sent_char_emb_p)
            char_cnn_max_out[:, sent_i, :], _ = torch.max(char_cnn_sent_out, dim=2)
        char_cnn = self.cnn_dropout(char_cnn_max_out)
        # concat word and char embedding
        # char_cnn_p = [sentence length, batch size, char emb dim * num filter]
        char_cnn_p = char_cnn.permute(1, 0, 2).to(device)
        word_features = torch.cat((embedding_out, char_cnn_p), dim=2)
        
        
        for i in range(self.N):
            x = self.layers[i](words, word_features, self.word_pad_idx)    
        return self.softmax(x)

    def init_embeddings(self, char_pad_idx=CHAR_PAD_IDX, word_pad_idx=TEXT_PAD_IDX,pretrained=None, freeze=True):
        # initialize embedding for padding as zero
        self.embedding.weight.data[self.word_pad_idx] = torch.zeros(self.embedding_dim)
        self.char_emb.weight.data[self.char_pad_idx] = torch.zeros(self.char_emb_dim)
        if pretrained is not None:
            self.embedding = nn.Embedding.from_pretrained(
                embeddings=torch.as_tensor(pretrained),
                padding_idx=self.word_pad_idx,
                freeze=freeze
            )
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

(nombre transformers=12,  Hidden size=768,  att_head=12,  Total  Param-eters=110M)  

`embedding_dim` + `char_emb_dim` * `char_cnn_filter_num` = `emb dim total`
e use a batch size of 32 and fine-tune for 3epochs over the data for all GLUE tasks. 
For eachtask, we selected the best fine-tuning learning rate(among 5e-5, 4e-5, 3e-5, and 2e-5) 

In [38]:
model = BERT(
    input_dim=len(TEXT.vocab),
    embedding_dim=300,
    char_emb_dim=45,  
    char_input_dim=len(CHAR.vocab),
    char_cnn_filter_num=4, 
    char_cnn_kernel_size=3,
    attn_heads=12,  
    fc_hidden=768,  
    trf_layers=1,
    output_dim=len(TAG.vocab),
    dropout=0.7,
    word_pad_idx=TEXT_PAD_IDX,
    char_pad_idx=CHAR_PAD_IDX,
    tag_pad_idx=TAG_PAD_IDX,
    N = 12
)
model.init_embeddings(
    char_pad_idx=CHAR_PAD_IDX,
    word_pad_idx=TEXT_PAD_IDX,
    pretrained= TEXT.vocab.vectors,
    freeze=True
)

print(f"Le modèle a {model.count_parameters():,} paramètres à entraîner.")

Le modèle a 39,276,774 paramètres à entraîner.


In [39]:
optimizer = optim.Adam(model.parameters(), lr = 5e-5)

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

model = model.to(device)
criterion = criterion.to(device)

In [40]:
def accuracy_per_tag(predictions, tags):
    n_tags = len(TAG.vocab)
    class_correct = list(0 for i in range(n_tags))
    class_total = list(0 for i in range(n_tags))
    acc = list(0 for i in range(n_tags))
    _, pred = torch.max(predictions, 1)
    # # compare predictions to true label
    correct = np.squeeze(pred.eq(tags.data.view_as(pred)))
    # # calculate test accuracy for each object class
    for i in range(len(tags.data)):
        label = tags.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
    for i in range(n_tags):
        if np.sum(class_total[i]) == 0 and np.sum(class_correct[i]) ==0:
            res = 100
        else:
            res = 100 * class_correct[i] / class_total[i]
        acc[i] = res, np.sum(class_correct[i]), np.sum(class_total[i])
        
    return acc  

from sklearn.metrics import f1_score
def f1_loss(preds, y, tag_pad_idx):
    index_o = TAG.vocab.stoi["O"]
    positive_labels = [i for i in range(len(TAG.vocab.itos))
                           if i not in (tag_pad_idx, index_o)]
    _, pred = torch.max(preds, 1)
    pred = pred.data.cpu().numpy() 
    tags = y.data.cpu().numpy()
    f1 = f1_score(
            y_true=tags,
            y_pred=pred,
            labels=positive_labels,
            average="micro"
        ) 
       
    return f1

In [41]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.tag
        chars = batch.char 
        optimizer.zero_grad()

        predictions = model(text,chars)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        f1 = f1_loss(predictions, tags, tag_pad_idx)
        acc = accuracy_per_tag(predictions, tags)   
        
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss.item()
        epoch_f1 += f1
        
    return epoch_loss / len(iterator), acc, epoch_f1 / len(iterator)

def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.tag
            chars = batch.char
            
            predictions = model(text,chars)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            acc = accuracy_per_tag(predictions, tags)
            f1 = f1_loss(predictions, tags, tag_pad_idx) 
            
            epoch_loss += loss.item()
            epoch_f1 += f1
        
    return epoch_loss / len(iterator), acc, epoch_f1 / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc, valid_f1 = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut7-model.pt')
    if epoch%2 == 0:
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train F1 score: {train_f1*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1 score: {valid_f1*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 29s
	Train Loss: 0.941 | Train F1 score: 0.15%
	 Val. Loss: 0.994 |  Val. F1 score: 0.00%
Epoch: 03 | Epoch Time: 2m 32s
	Train Loss: 0.813 | Train F1 score: 0.00%
	 Val. Loss: 0.912 |  Val. F1 score: 0.00%
Epoch: 05 | Epoch Time: 2m 28s
	Train Loss: 0.798 | Train F1 score: 0.00%
	 Val. Loss: 0.881 |  Val. F1 score: 0.00%


In [43]:
n_tags = len(TAG.vocab)
for i in range(n_tags):   
    print('Train Accuracy of %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], train_acc[i][0],
           train_acc[i][1], train_acc[i][2]))  

Train Accuracy of <pad>:  0% ( 0/776)
Train Accuracy of     O: 100% (295/295)
Train Accuracy of B-LOC:  0% ( 0/22)
Train Accuracy of B-PER:  0% ( 0/16)
Train Accuracy of B-ORG:  0% ( 0/ 8)
Train Accuracy of I-PER:  0% ( 0/11)
Train Accuracy of I-ORG:  0% ( 0/ 8)
Train Accuracy of B-MISC:  0% ( 0/ 7)
Train Accuracy of I-LOC:  0% ( 0/ 7)
Train Accuracy of I-MISC:  0% ( 0/ 2)


In [44]:
for i in range(n_tags):
    print('Valid Accuracy of %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], valid_acc[i][0],
           valid_acc[i][1], valid_acc[i][2]))
  

Valid Accuracy of <pad>:  0% ( 0/230)
Valid Accuracy of     O: 100% (148/148)
Valid Accuracy of B-LOC:  0% ( 0/ 1)
Valid Accuracy of B-PER:  0% ( 0/ 2)
Valid Accuracy of B-ORG:  0% ( 0/ 7)
Valid Accuracy of I-PER:  0% ( 0/ 2)
Valid Accuracy of I-ORG:  0% ( 0/ 5)
Valid Accuracy of B-MISC:  0% ( 0/ 1)
Valid Accuracy of I-LOC: 100% ( 0/ 0)
Valid Accuracy of I-MISC: 100% ( 0/ 0)


In [45]:
model.load_state_dict(torch.load('tut7-model.pt'))

test_loss, test_acc, test_f1 = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)
n_tags = len(TAG.vocab)
for i in range(n_tags):   
    print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], test_acc[i][0],
           test_acc[i][1], test_acc[i][2]))
print(f'Test Loss: {test_loss:.3f} |  Test F1 score: {test_f1*100:.2f}%')

Test Accuracy of <pad>:  0% ( 0/ 9)
Test Accuracy of     O: 100% (84/84)
Test Accuracy of B-LOC:  0% ( 0/ 4)
Test Accuracy of B-PER:  0% ( 0/ 1)
Test Accuracy of B-ORG: 100% ( 0/ 0)
Test Accuracy of I-PER: 100% ( 0/ 0)
Test Accuracy of I-ORG: 100% ( 0/ 0)
Test Accuracy of B-MISC:  0% ( 0/ 3)
Test Accuracy of I-LOC: 100% ( 0/ 0)
Test Accuracy of I-MISC:  0% ( 0/ 1)
Test Loss: 0.892 |  Test F1 score: 0.00%


In [21]:
def tag_sentence(model, device, sentence, text_field, tag_field, char_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    max_word_len = max([len(token) for token in tokens])
    numericalized_chars = []
    char_pad_id = char_field.vocab.stoi[CHAR.pad_token] 
    for token in tokens:
        numericalized_chars.append(
                [char_field.vocab.stoi[char] for char in token]
                + [char_pad_id for _ in range(max_word_len - len(token))]
                )
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
    char_tensor = torch.as_tensor(numericalized_chars)
    char_tensor = char_tensor.unsqueeze(0).to(device) 
    
    predictions = model(token_tensor, char_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

In [22]:
example_index = 6

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['tag']

print(sentence)
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       TEXT, 
                                       TAG,
                                       CHAR)

print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

['"', 'we', 'do', "n't", 'support', 'any', 'such', 'recommendation', 'because', 'we', 'do', "n't", 'see', 'any', 'grounds', 'for', 'it', ',', '"', 'the', 'commission', "'s", 'chief', 'spokesman', 'nikolaus', 'van', 'der', 'pas', 'told', 'a', 'news', 'briefing', '.']
Pred. Tag	Actual Tag	Correct?	Token

O		O		✔		"
O		O		✔		we
O		O		✔		do
O		O		✔		n't
O		O		✔		support
O		O		✔		any
O		O		✔		such
O		O		✔		recommendation
O		O		✔		because
O		O		✔		we
O		O		✔		do
O		O		✔		n't
O		O		✔		see
O		O		✔		any
O		O		✔		grounds
O		O		✔		for
O		O		✔		it
O		O		✔		,
O		O		✔		"
O		O		✔		the
O		B-ORG		✘		commission
O		O		✔		's
O		O		✔		chief
O		O		✔		spokesman
O		B-PER		✘		nikolaus
O		I-PER		✘		van
O		I-PER		✘		der
O		I-PER		✘		pas
O		O		✔		told
O		O		✔		a
O		O		✔		news
O		O		✔		briefing
O		O		✔		.


In [ ]:
sentence = 'The will deliver a speech about the conflict in Sao Paulo at tomorrow in Anne Mary with Jack.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                  TAG,
                                  CHAR)

print(unks)
print("Pred. Tag\tToken\n")


for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")